# __Brough Lecture Notes: GARCH Models - Asymmetric Models__

<br>

Finance 5330: Financial Econometrics <br>
Tyler J. Brough <br>
Last Updated: March 28, 2019 <br>
<br>
<br>

These notes are based in part on several chapters in the series of books on [Market Risk Analysis](http://carolalexander.org/market-risk-analysis/) by Carol Alexander.

<br>

There is a mind-numbing alphabet soup of variations on GARCH models. These notes introduce you to one simple extension known as the GJR-GARCH model, named for the initials of its [inventors](https://bit.ly/2U1d8Kj).

<br>

Ever since [Black 1976](https://bit.ly/2CGBNck) economists have become aware of what Black called the _leverage effect_.
This basically means that there is an _asymmetric_ response to past volatility shocks.

<br>

To account for this GJR add a single extra _leverage_ parameter that augments the weight of the response in the model to negative market shocks. 

<br>

The model can be written down as: 

<br>

$$
\sigma_{t}^{2} = \omega + \alpha \epsilon_{t-1}^{2} + \lambda I_{\{\epsilon_{t-1} < 0\}} \epsilon_{t-1}^{2} + \beta \sigma_{t-1}^{2}
$$

<br>

where $I_{\{\epsilon_{t-1} < 0 \}} = 1$ if $\epsilon_{t-1} < 0$ and $0$ otherwise (called an indicator function).

<br>

MLE parameter estimation is based on the usual GARCH likelihood function, where $\sigma_{t}$ depends now on $\lambda$.

<br>

__NB:__ Note that if $\lambda = 0$ then we have a simple GARCH(1,1) model, so the GJR-GARCH model is a generalization
        of the standard GARCH model. 
        
<br>

__NB:__ Note that a general GJR-GARCH(p, o, q) model could be written down similar to a GARCH(p, q) model where the $o$
        parameter is the number of lags in the _leverage term_. 
        
<br>

The GJR-GARCH model has often been found to improve the fit to data relative to a standard symmetric GARCH model. 

<br>

The standard model specification is a GRJ-GARCH(1,1,1) model (similar to the GARCH(1,1) case). 